## 08 베이지안 최적화 기반의 HyperOpt를 이용한 하이퍼 파라미터 튜닝

Grid Search 방식의 단점 : 튜닝해야 할 하이퍼 파라미터 개수가 많을 경우 최적화 수행 시간이 오래 걸린다. 개별 하이퍼 파라미터 값의 범위가 넓거나 학습 데이터가 대용량일 경우에는 최적화 시간이 더욱 늘어나게 된다. 

XGBoost나 LightGBM은 다른 알고리즘에 비해 많은 개수의 하이퍼 파라미터를 가지고 있다. 때문에 실무의 대용량 학습 데이터에 Grid Search 방식으로 최적 하이퍼 파라미터를 찾으려면 많은 시간이 소모될 수 있다. 

예시 ) LightGBM의 6가지 하이퍼 파라미터를 최적화하려고 시도

In [ ]:
params = {
'max_depth' = [10, 20, 30, 40, 50], 'num_leaves' = [35, 45, 55, 65],
'colsample_bytree' = [0.5, 0.6, 0.7, 0.8, 0.9], 'subsample' = [0.5, 0.6, 0.7, 0.8, 0.9],
'min_child_weight' = [10, 20, 30, 40], reg_alpha = [0.01, 0.05, 0.1]
}

이 경우 Grid Search 방식은 5×4×5×5×4×3 = 6000회에 걸쳐서 반복적으로 학습과 평가를 수행해야만 하기에 수행 시간이 매우 오래 걸릴 수 있다.

때문에 실무의 대용량 학습 데이터에 XGBoost나 LightGBM의 하이퍼 파라미터 튜닝 시에 Grid Search 방식보다는 다른 방식을 적용하곤 하는데 대표적으로 베지안 최적화 기법을 사용한다.

#### 베이지안 최적화 개요

베이지안 최적화는 목적 함수 식을 제대로 알 수 없는 블랙 박스 형태의 함수에서 최대 또는 최소 함수 반환 값을 만드는 최적 입력값을 가능한 적은 시도를 통해 빠르고 효과적으로 찾아주는 방식이다.

베이지안 최적화는 이름에서 알 수 있듯이 베이지안 확류에 기반을 두고 있는 최적화 기법이다. 베이지안 확률이 새로운 사건의 관측이나 새로운 샘플 데이터를 기반으로 사후 확률을 개선해 나가듯이, 베이지안 최적화는 새로운 데이터를 입력받았을 때 최적 함수를 예측하는 사후 모델을 개선해 나가면서 최적 함수 모델을 만든다.

베이지안 최적화를 구성하는 두 가지 중요 요소는 대체 모델(Surrogate Model)과 획득 함수(Acquisition Function)이다.

대체 모델은 획득 함수가 계산한 하이퍼 파라미터를 입력받으면서 점차적으로 개선되며, 개선된 대체 모델을 기반으로 획득 함수는 더 정확한 하이퍼 파라미터를 계산할 수 있게 된다.

__베이지안 최적화 과정__

+ step 1 : 최초에는 랜덤하게 하이퍼 파라미터들을 샘플링하고 성능 결과를 관측한다. 아래 그림에서 검은색 원은 특정 하이퍼 파라미터가 입력되었을 때 관측된 성능 지표 결괏값을 뜻하며 주황색 사선은 찾아야 할 목표 최적함수이다.

&nbsp;
 
+ step 2 : 관측된 값을 기반으로 대체 모델은 최적 함수를 추정한다. 아래 그림에서 파란색 실선은 대체 모델이 추정한 최적 함수이다. 아래 그림에서 파란색 실선은 대체 모델이 추정한 최적 함수이다. 옅은 파란색으로 되어 있는 영역은 예측된 함수의 신뢰 구간이다. 추정된 함수의 결괏값 오류 편차를 의미하며 추정 함수의 불확실성을 나타낸다. 최적 관측값은 y축 value에서 가장 높은 값을 가질 때의 하이퍼 파라미터이다.

&nbsp;
 
+ setp 3 : 추정된 최적 함수를 기반으로 획득 함수(Acquisition Function)는 다음으로 관측할 하이퍼 파라미터 값을 계산한다. 획득 함수는 이전의 최적 관측값보다 더 큰 최댓값을 가질 가능성이 높은 지점을 찾아서 다음에 관측할 하이퍼 파라미터를 대체 모델에 전달한다.

&nbsp;

+ step 4 : 획득 함수로부터 전달된 하이퍼 파라미터를 수행하여 관측된 값을 기반으로 대체 모델은 갱신되어 다시 최적 함수를 예측 추정한다.

&nbsp;

이런 방식으로 step3과 step 4를 특정 횟수만큼 반복하게 되면 대체 모델의 불확실성이 개선되고 점차 정확한 최적 함수 추정이 가능하게 된다.

대체 모델은 최적 함수를 추정할 때 다양한 알고리즘을 사용할 수 있는데 일반적으로는 가우시안 프로세스(Gaussian Process)를 적용한다. 하지만 뒤에 나올 HyperOpt는 가우시안 프로세스가 아닌 트리 파르젠 Estimator(TPE, Tree-structure Parzen Estimator)를 사용한다.

### HyperOpt  사용하기

HyperOpt, Bayesian Optimization, Optuna 등의 파이썬 패키지를 사용하여 베이지안 최적화를 머신러닝 모델의 하이퍼 파라미터 튜닝에 적용할 수 있다.

__HyperOpt 설치__

명령 프롬프트에 `pip install hyperopt` 입력

__HyperOpt를 활용하는 주요 로직__

+ 첫째는 입력 변수명과 입력값의 검색 공간(Search Space) 설정이다.

+ 둘째는 목적 함수(Objective Function)의 설정이다.

+ 마지막으로 `목적 함수의 반환 최솟값을 가지는 최적 입력값을 유추`하는 것이다.

--------------------------------
__HyperOpt의 hp 모듈을 이용하여 입력 변수명과 입력값의 검색 공간 설정하기__

입력 변수명과 입력값 검색 공간은 파이썬 딕셔너리 형태로 설정되어야 하며, 키(key)값으로 입력 변수명, 밸류(value)값으로 해당 입력 변수의 검색 공간이 주어진다.

In [2]:
from hyperopt import hp

# -10 ~ 10까지 1간격을 가지는 입력 변수 x와 -15~15까지 1간격으로 입력 변수 y 설정
search_space = {'x' : hp.quniform('x', -10, 10, 1), 'y' : hp.quniform('y', -15, 15, 1)}

hp 모듈은 입력값의 검색 공간을 다양하게 설정할 수 있도록 여러 가지 함수를 제공한다.

+ hp.quniform(label, low, high, q) : label로 지정된 입력값 변수 검색 공간을 최솟값 low에서 최댓값 high까지 q의 간격을 가지고 설정

+ hp.uniform(label, low, high) : 최솟값 low에서 최댓값 high까지 정규 분포 형태의 검색 공간 설정

+ hp.randint(label, upper) : 0 부터 최댓값 upper까지 random한 정숫값으로 검색 공간 설정.

+ hp.loguniform(label, low, high) : exp(uniform(low, high)) 값을 반환하며, 반환 값의 log 변환 된 값은 정규 분포 형태를 가지는 검색 공간 설정

+ hp.choice(label, options) : 검색 값이 문자열 또는 문자열과 숫자값이 섞여 있을 경우 설정. Options는 리스트나 튜플 형태로 제공되며 hp.choice('tree_criterion', ['gini', 'entropy'])과 같이 설정하면 입력 변수 tree_criterion의 값을 'gini'와 'entropy'로 설정하여 입력함

__목적 함수 생성__

목적 함수는 반드시 변숫값과 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환하는 구조로 만들어져야 한다.

In [15]:
from hyperopt import STATUS_OK

# 목적 함수를 생성, 변숫값과 변수 검색 공간을 가지는 딕셔너리를 인자로 받고, 특정 값을 반환
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y
    
    return retval

위 코드는 search_space로 지정된 딕셔너리에서 x 입력 변숫값과 y 입력 변숫값을 추출하여 retval = x²- 20y로 계산된 값을 반환한다. 목적 함수의 반환값은 숫자형 단일값 외에도 딕셔너리 형태로 반환할 수 있다. 딕셔너리 형태로 반환할 경우에는 {'loss':retval, 'status':STATUS_OK}와 같이 loss와 status 키 값을 설정해서 반환해야한다.

입력값의 검색 공간과 목적 함수를 설정했으면 목적 함수의 반환값이 최소가 될 수 있는 최적의 입력값을 베이지안 최적화 기법에 기반하여 찾아 줘야 한다. HyperOpt는 이러한 기능을 수행할 수 있도록 fmin(objective, space, algo, max_evals, trials) 함수를 제공한다. fmin() 함수의 주요 인자는 아래와 같다.

+ fn : 위에서 생성한 objective_func와 같은 목적 함수이다.

+ space : 위에서 생성한 search_space와 같은 검색 공간 딕셔너리이다.

+ algo : 베이지안 최적화 적용 알고리즘이다. 기본적으로 tpe.suggest이며 이는 HyperOpt의 기본 최적화 알고리즘인 TPE(Tree of Parzen Estimatr)를 의미한다.

+ max_evals : 최적 입력값을 찾기 위해 시도한 입력값 및 해당 입력값의 목적 함수 반환값 결과를 저장하는 데 사용된다. Trials 클래스를 객체로 생성한 변수명을 입력한다.

+ rstate : fmin()을 수행할 때마다 동일한 결괏값을 가질 수 있도록 설정하는 랜덤 시드(seed) 값이다.

__위에서 설정한 검색 공간인 search_space에서 목적 함수 object_func의 최솟값을 반환하는 최적 입력 변숫값을 찾을 수 있도록 베이지안 최적화 수행__

HyperOpt의 fmin() 함수를 호출하되, 먼저 5번의 입력값 시도로 찾아낼 수 있도록 max_evals 인자 값으로 5를 설정

목적 함수인 fn 인자로는 objective_func를, 검색 공간 인자인 space에는 search_space를, 최적화 적용 알고리즘 algo 인자는 기본값인 tpe.suggest로 설정

또한 trials 인자값으로는 Trials() 객체를, 그리고 rstate의 경우는 임의의 랜덤 시드값을 입력

일반적으로는 rstate를 잘 적용하지 않으며 HyperOpt는 rstate에 넣어주는 인자값으로 일반적인 정수형 값을 넣지 않는다. 또한 버전별로 rstate 인자값이 조금씩 다르다. 0.2.7버전에서는 넘파이의 random Generator를 생성하는 random.default_rng() 함수 인자로 seed값을 입력하는 방식이다.

In [16]:
from hyperopt import fmin, tpe, Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체값 생성
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_evals=5)로 찾아냄.
best_01 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=5, 
               trials=trial_val, rstate=np.random.default_rng(seed=0))

print('best: ', best_01)

100%|█████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 238.69trial/s, best loss: -224.0]
best:  {'x': -4.0, 'y': 12.0}


입력 변수 x의 공간 -10 ~10, y의 공간 -15 ~15에서 목적 함수의 반환 값을 x²- 20y로 설정했으므로 x는 0에 가까울수록 y는 15에 가까울수록 반환값이 최소로 근사될 수 있다. 확실하게 만족할 수준의 최적 x와 y값을 찾은 것은 아니지만, 5번의 수행으로 어느 정도 최적값에 다가설 수 있었다는 점을 알 수 있다.

__max_evals 값을 20으로 설정하여 20번의 수행으로 어떤 최적값을 반환하는지 살펴보기__

In [17]:
trial_val = Trials()

# max_evals 를 20회로 늘려서 재테스트
best_02 = fmin(fn=objective_func, space=search_space, algo=tpe.suggest, max_evals=20,
               trials=trial_val, rstate=np.random.default_rng(seed=0))

print('best: ', best_02)

100%|███████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 232.55trial/s, best loss: -296.0]
best:  {'x': 2.0, 'y': 15.0}


20회 반복시 x는 2로, y는 15로 목적 함수의 최적 최솟값을 근사할 수 있는 결과를 도출했다. 

완벽한 정답인 x=0은 도출하지 못했지만, 입력값 x가 -10 ~ 10까지 21개의 경우의 수, 입력값 y가 -15~15까지 31개의 경우의 수를 가질 수 있기에, 만일 그리드 서치와 같이 순차적으로 x,y 변숫값을 입력해서 최소 함수 반환값을 찾는다면 최대 21*31=651회의 반복이 필요할 수도 있는데 반해서, 베이지안 최적화를 이용해서는 20회의 반복만으로 일정 수준의 최적값을 근사해 낼 수 있었다.

__fmin() 함수 수행 시 인자로 들어가는 Trials 객체의 중요 속성 results 와 value__

+ results 속성 : 함수의 반복 수행 시마다 반환되는 반환값을 가진다. 파이썬 리스트 형태이며 리스트 내의 개별 원소는 {'loss':함수 반환값, 'status':반환 상태값}과 같은 딕셔너리로 가지고 있다. 

+ value 속성 : 함수의 반복 수행 시마다 입력되는 입력 변숫값을 가진다. 딕셔너리 형태로 값을 가진다. 

In [18]:
# fmin()에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장됨
# 리스트 내부의 개별 원소는 {'loss':함수 반환값, 'status':반환 상태값}와 같은 딕셔너리임.
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


max_evals=20으로 fmin() 함수는 20회의 반복 수행을 했으므로 results 속성은 loss와 status를 키값으로 ㄱ자니느 20개의 딕셔너리를 개별 원소를 가지는 리스트로 구성되어 있음을 알 수 있다.

In [19]:
# Trials 객체의 vals 속성에 {'입력변수명':개별 수행 시마다 입력된 값 리스트} 형태로 저장됨.
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


vals는 딕셔너리 형태의 값을 가지며, 입력 변수 x와 y를 키값으로 가지며, x와 y키 값의 밸류는 20회의 반복 수행 시마다 사용되는 입력값들을 리스트 형태로 가지고 있는 것을 알 수 있다.

In [20]:
# results와 vals 속성값들을 DataFrame으로 만들어 직관적으로 보기

import pandas as pd

# results에서 loss 키값에 해당하는 밸류들을 추출하여 list로 생성. 
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성.
result_df = pd.DataFrame({'x': trial_val.vals['x'], 'y': trial_val.vals['y'], 'losses': losses})
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


### HyperOpt를 이용한 XGBoost 하이퍼 파라미터 최적화

적용해야 할 하이퍼 파라미터와 검색 공간을 설정하고, 목적 함수에서 XGBoost를 학습 후에 예측 성능 결과를 반환 값으로 설정한다. 그리고 fmin() 함수에서 목적 함수를 하이퍼 파라미터 검색 공간의 입력값들을 사용하여 최적의 예측 성능 결과를 반환하는 최적 입력값들을 결정하는 것이다. 

+ 주의 할 점

특정 하이퍼 파라미터들은 정숫값만 이력을 받는데 HyperOpt는 입력값과 반환 값이 모두 실수형이기 대문에 하이퍼 파라미터 입력 시 형변환을 해줘야 하는 부분, 그리고 HyperOpt의 목적 함수는 최솟값을 반환할 수 있도록 최적화해야 하기 때문에 성능 값이 클수록 좋은 성능 지표일 경우 -1을 곱해줘야 한다. 

In [21]:
# 위스콘신 유방암 데이터 세트 로드
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
cancer_df['target']= dataset.target
X_features = cancer_df.iloc[:, :-1]
y_label = cancer_df.iloc[:, -1]

In [22]:
# 전체 데이터 중 80%는 학습용 데이터, 20%는 테스트용 데이터 추출
X_train, X_test, y_train, y_test=train_test_split(X_features, y_label, test_size=0.2, random_state=156 )

# 앞에서 추출한 학습 데이터를 다시 학습과 검증 데이터로 분리
X_tr, X_val, y_tr, y_val= train_test_split(X_train, y_train, test_size=0.1, random_state=156 )

__하이퍼 파라미터와 검색공간 설정__

max_depth는 5에서 20까지 1간격으로 (정수형 하이퍼 파라미터 이므로 hp.quniform() 사용 )

min_child_weight는 1에서 2까지 1간격으로 (정수형 하이퍼 파라미터 이므로 hp.quniform() 사용 )

learning_rate는 0.01에서 0.2사이 정규 분포된 값으로 (hp.uniform() 사용)

colsample_bytree는 0.5에서 1사이 정규 분포된 값으로 하이퍼 파라미터 검색 공간을 설정  (hp.uniform() 사용)

In [23]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로, min_child_weight는 1에서 2까지 1간격으로
# colsample_bytree는 0.5에서 1사이, learning_rate는 0.01에서 0.2 사이 정규 분포된 값으로 검색.
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1), 
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1),
                   }

__목적 함수 설정하기__

하이퍼 파라미터 튜닝을 위한 목적 함수는 검색 공간에서 설정한 하이퍼 파라미터들을 입력받아서 XGBoost를 학습하고 평가지표를 반환할 수 있도록 구성한다.

+ 유의 사항

1. 검색 공간에서 목적 함수로 입력되는 모든 인자들은 실수형 값이므로 이들을 XGBoostClassifier의 정수형 하이퍼 파라미터값으로 설정할 때는 정수형으로 형변환을 해야 한다. 즉, XGBoostClassifier(max_depth=int(search_space['max_depth'])) 같이 정수형 값으로 형변환을 해서 하이퍼 파라미터로 입력해 주어야 한다.

2. HyperOpt의 목적함수는 최솟값을 반환할 수 있도록 최적화해야 하기 때문에 정확도와 같이 값이 클수록 좋은 성능 지표일 경우 -1을 곱한 뒤 반환하여, 더 큰 성능 지표가 더 작은 반환값이 되도록 만들어 줘야 한다. MAE,RMSE와 같은 지표는 작을수록 좋기 때문에 반환 시 -1을 곱해줄 필요가 없다.

목적 함수인 objective_func() 정의

목적 함수의 반환값은 교차 검증 기반의 평균 정확도(accuracy)를 사용한다. 3개의 교차 검증 세트로 정확도를 반환할 수 있도록 cross_val_score()를 적용하며, 수행 시간을 줄이기 위해서 n_estimators는 100으로 제한한다. cross_val_score()를 XGBoost나 LightGBM에 적용할 경우 조기 중단(early stopping)이 지원되지 않는다.

In [24]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에서 입력된 search_space 값으로 입력된 모든 값은 실수형임.
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함.
# 정확도는 높을수록 더 좋은 수치임. -1 * 정확도를 곱해서 큰 정확도 값일수록 최소가 되도록 변환
def objective_func(search_space):
    # 수행 시간 절약을 위해 nestimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'],
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    
    # accuracy는 cv=3 개수만큼 roc-auc 결과를 리스트로 가짐. 이를 평균해서 반환하되 -1을 곱함.
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

__fmin()을 이용해 최적 하이퍼 파라미터를 도출하기__

최대 반복 수행 횟수 max_evals는 50회 지정하고 앞에서 설정한 목적함수 objective_func, 하이퍼 파라미터 검색공간 xgb_search_space 등을 인자로 입력한다. 동일한 실습결과를 도출하기 위해 rstate는 np.random.de-fault_rng(seed=9))로 설정

In [25]:
from hyperopt import fmin, tpe, Trials

trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [00:27<00:00,  1.80trial/s, best loss: -0.9670616939700244]
best: {'colsample_bytree': 0.5424149213362504, 'learning_rate': 0.12601372924444681, 'max_depth': 17.0, 'min_child_weight': 2.0}


정수형 하이퍼 파라미터인 max_depth, min_child_weight가 실수형 값으로 도출되었음을 유의

__fmin()으로 추출된 최적 하이퍼 파라미터를 직접 XGBClassifier에 인자로 입력하기 전에 정수형 하이퍼 파라미터는 정수형으로 형 변환을, 실수형 하이퍼 파라미터는 소수점 5자리까지만 변환 후 확인__해보기

In [26]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
    round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
    int(best['max_depth']), int(best['min_child_weight'])))

colsample_bytree:0.54241, learning_rate:0.12601, max_depth:17, min_child_weight:2


__된 최적 하이퍼 파라미터들을 이용해서 XGBClassifier를 재학습한 후 성능 평가 결과를 확인하기__

XGBoost의 조기 중단을 검증 데이터 세트로 활용하며 n_estimators는 4000으로 증가시킨다. get_clf_eval()을 이용해 성능 평가를 한다.

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))


In [28]:
xgb_wrapper = XGBClassifier(n_estimators=400,
                            learning_rate=round(best['learning_rate'], 5),
                            max_depth=int(best['max_depth']),
                            min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )

evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr, y_tr, early_stopping_rounds=50, eval_metric='logloss',
                eval_set=evals, verbose=True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:, 1]

get_clf_eval(y_test, preds, pred_proba)

[0]	validation_0-logloss:0.58942	validation_1-logloss:0.62048
[1]	validation_0-logloss:0.50801	validation_1-logloss:0.55913
[2]	validation_0-logloss:0.44160	validation_1-logloss:0.50928
[3]	validation_0-logloss:0.38734	validation_1-logloss:0.46815
[4]	validation_0-logloss:0.34224	validation_1-logloss:0.43913
[5]	validation_0-logloss:0.30425	validation_1-logloss:0.41570
[6]	validation_0-logloss:0.27178	validation_1-logloss:0.38953
[7]	validation_0-logloss:0.24503	validation_1-logloss:0.37317
[8]	validation_0-logloss:0.22050	validation_1-logloss:0.35628
[9]	validation_0-logloss:0.19873	validation_1-logloss:0.33798
[10]	validation_0-logloss:0.17945	validation_1-logloss:0.32463
[11]	validation_0-logloss:0.16354	validation_1-logloss:0.31384
[12]	validation_0-logloss:0.15032	validation_1-logloss:0.30607
[13]	validation_0-logloss:0.13813	validation_1-logloss:0.30143
[14]	validation_0-logloss:0.12798	validation_1-logloss:0.29513
[15]	validation_0-logloss:0.11926	validation_1-logloss:0.28891
[1

[130]	validation_0-logloss:0.01639	validation_1-logloss:0.22934
[131]	validation_0-logloss:0.01634	validation_1-logloss:0.22987
[132]	validation_0-logloss:0.01629	validation_1-logloss:0.22927
[133]	validation_0-logloss:0.01624	validation_1-logloss:0.23076
[134]	validation_0-logloss:0.01620	validation_1-logloss:0.23030
[135]	validation_0-logloss:0.01615	validation_1-logloss:0.22891
[136]	validation_0-logloss:0.01610	validation_1-logloss:0.22883
[137]	validation_0-logloss:0.01606	validation_1-logloss:0.22882
[138]	validation_0-logloss:0.01602	validation_1-logloss:0.22876
[139]	validation_0-logloss:0.01597	validation_1-logloss:0.22734
[140]	validation_0-logloss:0.01592	validation_1-logloss:0.22881
[141]	validation_0-logloss:0.01588	validation_1-logloss:0.22935
[142]	validation_0-logloss:0.01583	validation_1-logloss:0.22880
[143]	validation_0-logloss:0.01579	validation_1-logloss:0.22856
[144]	validation_0-logloss:0.01575	validation_1-logloss:0.22725
[145]	validation_0-logloss:0.01571	valid